In [ ]:
# !pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2
# !pip install llama-cpp-python

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# Load Model

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map = "cuda",
    torch_dtype = "auto",
    trust_remote_code = False
)

In [ ]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

In [5]:
# Create a Pipeline

piep = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,


    return_full_text = False,
    max_new_tokens = 500,
    do_sample = False
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [6]:
message = [
    {"role": "user",
     "content": "Create a funny joke about 1st day of new year."}
]

In [7]:
output = piep(message)
print(output[0]["generated_text"])

 Why don't scientists trust atoms on the 1st day of the new year?

Because they make up everything, even the resolutions!


In [8]:
# Apply prompt template
prompt = piep.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)

print(prompt)

<|user|>
Create a funny joke about 1st day of new year.<|end|>
<|assistant|>



In [9]:
# Apply prompt template, tokenize=True menas return Token ID
prompt = piep.tokenizer.apply_chat_template(message, tokenize=True, add_generation_prompt=True)

print(prompt)

[32010, 6204, 263, 2090, 1460, 2958, 446, 1048, 29871, 29896, 303, 2462, 310, 716, 1629, 29889, 32007, 32001]


In [10]:
# Using a high temperature

output = piep(message, do_sample=True, temperature=1)
print(output[0]["generated_text"])

 Why don't scientists trust atoms on the 1st day of the new year? 

Because they make up everything, even the "new year's resolutions!"


In [11]:
output = piep(message, do_sample=True, temperature=1)
print(output[0]['generated_text'])

 Why don't scientists trust atoms on the first day of the new year?

Because they make up everything! Even the "joy" of starting fresh is just a "particle" of illusion.


### Intro to Prompt Engineering

#### The Basic Ingredients of a Prompt

### Advanced Prompt Engineering

#### Comples Prompt

In [12]:
# Text to summarize which we stole from https://jalammar.github.io/illustrated-transformer/ ;)
text = """In the previous post, we looked at Attention – a ubiquitous method in modern deep learning models. Attention is a concept that helped improve the performance of neural machine translation applications. In this post, we will look at The Transformer – a model that uses attention to boost the speed with which these models can be trained. The Transformer outperforms the Google Neural Machine Translation model in specific tasks. The biggest benefit, however, comes from how The Transformer lends itself to parallelization. It is in fact Google Cloud’s recommendation to use The Transformer as a reference model to use their Cloud TPU offering. So let’s try to break the model apart and look at how it functions.
The Transformer was proposed in the paper Attention is All You Need. A TensorFlow implementation of it is available as a part of the Tensor2Tensor package. Harvard’s NLP group created a guide annotating the paper with PyTorch implementation. In this post, we will attempt to oversimplify things a bit and introduce the concepts one by one to hopefully make it easier to understand to people without in-depth knowledge of the subject matter.
Let’s begin by looking at the model as a single black box. In a machine translation application, it would take a sentence in one language, and output its translation in another.
Popping open that Optimus Prime goodness, we see an encoding component, a decoding component, and connections between them.
The encoding component is a stack of encoders (the paper stacks six of them on top of each other – there’s nothing magical about the number six, one can definitely experiment with other arrangements). The decoding component is a stack of decoders of the same number.
The encoders are all identical in structure (yet they do not share weights). Each one is broken down into two sub-layers:
The encoder’s inputs first flow through a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. We’ll look closer at self-attention later in the post.
The outputs of the self-attention layer are fed to a feed-forward neural network. The exact same feed-forward network is independently applied to each position.
The decoder has both those layers, but between them is an attention layer that helps the decoder focus on relevant parts of the input sentence (similar what attention does in seq2seq models).
Now that we’ve seen the major components of the model, let’s start to look at the various vectors/tensors and how they flow between these components to turn the input of a trained model into an output.
As is the case in NLP applications in general, we begin by turning each input word into a vector using an embedding algorithm.
Each word is embedded into a vector of size 512. We'll represent those vectors with these simple boxes.
The embedding only happens in the bottom-most encoder. The abstraction that is common to all the encoders is that they receive a list of vectors each of the size 512 – In the bottom encoder that would be the word embeddings, but in other encoders, it would be the output of the encoder that’s directly below. The size of this list is hyperparameter we can set – basically it would be the length of the longest sentence in our training dataset.
After embedding the words in our input sequence, each of them flows through each of the two layers of the encoder.
Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer.
Next, we’ll switch up the example to a shorter sentence and we’ll look at what happens in each sub-layer of the encoder.
Now We’re Encoding!
As we’ve mentioned already, an encoder receives a list of vectors as input. It processes this list by passing these vectors into a ‘self-attention’ layer, then into a feed-forward neural network, then sends out the output upwards to the next encoder.
"""

In [13]:
# Prompt components
persona = "You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.\n"
instruction = "Summarize the key findings of the paper provided.\n"
context = "Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.\n"
data_format = "Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.\n"
audience = "The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.\n"
tone = "The tone should be professional and clear.\n"
text = "MY TEXT TO SUMMARIZE"  # Replace with your own text to summarize
data = f"Text to summarize: {text}"

In [14]:
# The full prompt - remove and add pieces to view its impact on the generated output
query = persona + instruction + context + data_format + audience + tone + data

In [15]:
message = [
    {'role': 'user',
     'content': query}
]

In [16]:
print(tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True))

<|user|>
You are an expert in Large Language models. You excel at breaking down complex papers into digestible summaries.
Summarize the key findings of the paper provided.
Your summary should extract the most crucial points that can help researchers quickly understand the most vital information of the paper.
Create a bullet-point summary that outlines the method. Follow this up with a concise paragraph that encapsulates the main results.
The summary is designed for busy researchers that quickly need to grasp the newest trends in Large Language Models.
The tone should be professional and clear.
Text to summarize: MY TEXT TO SUMMARIZE<|end|>
<|assistant|>



In [17]:
outputs = piep(message)
print(outputs[0]['generated_text'])

 - The paper investigates the impact of pre-training data size on the performance of Large Language Models (LLMs).

- It compares models trained on different volumes of data, ranging from small to extra-large datasets.

- The study finds that models trained on larger datasets generally perform better on a variety of tasks.

- However, the performance gains diminish as the dataset size increases beyond a certain point.

- The paper also explores the cost-benefit trade-off of using larger datasets for training LLMs.

- It concludes that there is an optimal dataset size that balances performance gains with training costs.


In summary, the paper presents a comprehensive analysis of how the size of pre-training data affects the performance of Large Language Models. The researchers found that while larger datasets tend to yield better model performance across various tasks, the benefits plateau beyond a certain dataset size. This indicates that there is a point of diminishing returns when i

### In-Context Learning: Providing Examples

In [18]:
# Use a single example of using the made-up word in a sentence
one_shot_prompt = [
    {
        "role": "user",
        "content": "A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:"
    },
    {
        "role": "assistant",
        "content": "I have a Gigamuru that my uncle gave me as a gift. I love to play it at home."
    },
    {
        "role": "user",
        "content": "To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word screeg is:"
    }
]

In [19]:
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False, add_generation_prompt=True))

<|user|>
A 'Gigamuru' is a type of Japanese musical instrument. An example of a sentence that uses the word Gigamuru is:<|end|>
<|assistant|>
I have a Gigamuru that my uncle gave me as a gift. I love to play it at home.<|end|>
<|user|>
To 'screeg' something is to swing a sword at it. An example of a sentence that uses the word screeg is:<|end|>
<|assistant|>



In [20]:
# Generate the Output

outputs = piep(one_shot_prompt, do_sample=True)
print(outputs[0]['generated_text'])

 At the historical reenactment, the knights screeged their foes with precision, showcasing their mastery in medieval combat techniques.


### Chain Prompting: Breaking up the Problem

In [21]:
# Create name and slogan for a product
product_prompt = [
    {'role': 'user',
     'content': 'Create a name and slogan for a chatbot that leverages LLMs.'}
]

output = piep(product_prompt, do_sample=True)
product_description = output[0]['generated_text']
print(product_description)

 Name: ChatBot AI

Slogan: "Elevating Conversations with Cutting-Edge AI"


In [22]:
# Based on a name and slogan for a product, generate a sales pitch
sales_prompt = [
    {"role": "user", "content": f"Generate a very short sales pitch for the following product: '{product_description}'"}
]
outputs = piep(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

 Introducing ChatBot AI, the future of engaging dialogue. With our cutting-edge AI technology, we're not just elevating conversations; we're redefining them. Experience seamless interaction and unparalleled assistance with ChatBot AI – where every chat is a step towards innovation.


## Reasoning with Generative Models ##

### Chain-of-Thought: Think Before Answering ###

In [23]:
# Answering with chain-of-thought
cot_prompt = [
    {"role": "user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
    {"role": "assistant", "content": "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."},
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}
]

# Generate the output
outputs = piep(cot_prompt)
print(outputs[0]["generated_text"])

 The cafeteria started with 23 apples. They used 20 apples to make lunch, so they had 23 - 20 = 3 apples left. After buying 6 more apples, they now have 3 + 6 = 9 apples. The answer is 9.


### Zero Short Chain of Thought ###

In [25]:
# Zero-shot Chain-of-Thought
zeroshot_cot_prompt = [
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? Let's think step-by-step."}
]

# Generate the output
outputs = piep(zeroshot_cot_prompt)
print(outputs[0]["generated_text"])

 Step 1: The cafeteria starts with 23 apples.
Step 2: They used 20 apples to make lunch, so we subtract 20 from the initial amount: 23 - 20 = 3 apples remaining.
Step 3: The cafeteria bought 6 more apples, so we add 6 to the remaining amount: 3 + 6 = 9 apples.

The cafeteria now has 9 apples.


### Tree-of-Thought: Exploring Intermediate Steps ###

In [26]:
# Zero-shot Chain-of-Thought
zeroshot_tot_prompt = [
    {"role": "user", "content": "Imagine three different experts are answering this question. All experts will write down 1 step of their thinking, then share it with the group. Then all experts will go on to the next step, etc. If any expert realises they're wrong at any point then they leave. The question is 'The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?' Make sure to discuss the results."}
]

In [28]:
output = piep(zeroshot_tot_prompt)
print(output[0]['generated_text'])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 Expert 1:
Step 1: Start with the initial number of apples, which is 23.

Expert 2:
Step 1: Subtract the number of apples used for lunch, which is 20.
Step 2: Add the number of apples bought, which is 6.

Expert 3:
Step 1: Start with the initial number of apples, which is 23.
Step 2: Subtract the number of apples used for lunch, which is 20.
Step 3: Add the number of apples bought, which is 6.

Results:
All three experts arrived at the same answer: 23 - 20 + 6 = 9 apples remaining in the cafeteria.
